## Libraries

In [ ]:
!pip install -Uq openai-agents "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


## Setup & Imports

In [ ]:
import asyncio
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    output_guardrail,
    set_tracing_disabled
)
from agents.extensions.models.litellm_model import LitellmModel
from pydantic import BaseModel

import nest_asyncio
nest_asyncio.apply()

set_tracing_disabled(True)

import os
from google.colab import userdata
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
API_KEY = userdata.get('GEMINI_API_KEY')

## Agent

In [ ]:
class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str
    answer: str

In [ ]:
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the input given by user is related to mathematics.",
    model=LitellmModel(model="gemini/gemini-2.0-flash"),
    output_type=MathHomeworkOutput,
)

In [ ]:
print("----------------------\n")

result = Runner.run_sync(
    starting_agent=guardrail_agent,
    input="What is 2+2?",
)

print(result.final_output.model_dump())

print("\n----------------------\n")

result = Runner.run_sync(
    starting_agent=guardrail_agent,
    input="What is Indulge meaning?",
)

print(result.final_output.model_dump())

print("\n----------------------")

----------------------

{'is_math_homework': True, 'reasoning': 'The question is a basic arithmetic addition problem.', 'answer': '4'}

----------------------

{'is_math_homework': False, 'reasoning': "The query asks for the definition of the word 'Indulge'. This is a vocabulary question and has nothing to do with mathematics.", 'answer': 'The query is not related to mathematics.'}

----------------------


## Input Guardrail

In [ ]:
@input_guardrail
async def math_input_guardrail(ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:
  result = await Runner.run(guardrail_agent, input, context=ctx.context)

  print(f"Is math related question: {result.final_output.is_math_homework}")

  return GuardrailFunctionOutput(
      output_info=result.final_output,
      tripwire_triggered=result.final_output.is_math_homework,
  )

assistant_agent = Agent(
      name="Assistant",
      instructions="You are a helpful assistant.",
      model=LitellmModel(model="gemini/gemini-2.0-flash"),
      input_guardrails=[math_input_guardrail],
  )


In [ ]:
print("----------------------\n")

try:
  result = Runner.run_sync(assistant_agent, "What is 2+2?")
  print(result.final_output)
except InputGuardrailTripwireTriggered as e:
  print(f"Error --> {e}")

print("\n----------------------\n")

try:
  result = Runner.run_sync(assistant_agent, "What is the meaning of Indulge?")
  print(result.final_output)
except InputGuardrailTripwireTriggered as e:
  print(f"Error --> {e}")

print("\n----------------------")

----------------------

Is math related question: True
Error --> Guardrail InputGuardrail triggered tripwire

----------------------

Is math related question: False
The word "indulge" has a few related meanings, but they all center around allowing oneself or someone else to have something enjoyable, often to excess. Here's a breakdown:

*   **To allow oneself to enjoy the pleasure of:** This is perhaps the most common meaning. It means to allow yourself to have something you want, even if it's not strictly necessary or even good for you. Examples: "I indulged in a slice of chocolate cake," or "She indulged in a long, hot bath."

*   **To allow (someone) to have something desired:** This means to give someone else something they want, even if it's not strictly necessary. Examples: "They indulged their children with expensive toys," or "The teacher indulged the student's request to use a pen instead of a pencil."

*   **To become involved in (an activity, typically one that is undesirab